In [1]:
pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp37-none-any.whl size=5887 sha256=ed8ea1b562aebd08605021af8cf89bb1f624cca16543e8ac9f0d3c45a5db9c0f
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [3]:
import os
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc
from scipy.io import wavfile

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization

from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn import metrics 


In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
def get_file_from_drive(folder_id, file_name):
  file_list = drive.ListFile({'q': "'" + folder_id + "' in parents and trashed=false"}).GetList()
  for file in file_list:
    if file['title'] == file_name:
      return file['id']

In [6]:
drive_folder_id='1r68SFGZVQuNQyy8ur8grGLDlQRmeoBzp'

def upload_data_system():
  downloaded = drive.CreateFile({'id': get_file_from_drive(drive_folder_id, 'x_train_filename.npy')})
  downloaded.GetContentFile('x_train_filename.npy') 

  downloaded = drive.CreateFile({'id': get_file_from_drive(drive_folder_id, 'x_test_filename.npy')})
  downloaded.GetContentFile('x_test_filename.npy')

  downloaded = drive.CreateFile({'id': get_file_from_drive(drive_folder_id, 'y_train_labels.npy')})
  downloaded.GetContentFile('y_train_labels.npy')

  downloaded = drive.CreateFile({'id': get_file_from_drive(drive_folder_id, 'y_test_labels.npy')})
  downloaded.GetContentFile('y_test_labels.npy')

upload_data_system()

In [27]:
dset_train_in = '/content/drive/MyDrive/ASC/Phase_3/Audio_Dataset/Train/Indoor'
dset_train_out = '/content/drive/MyDrive/ASC/Phase_3/Audio_Dataset/Train/Outdoor'
dset_test_in ='/content/drive/MyDrive/ASC/Phase_3/Audio_Dataset/Test/Indoor'
dset_test_out ='/content/drive/MyDrive/ASC/Phase_3/Audio_Dataset/Test/Outdoor'

#load npy filenames
x_train_filename = np.load('/content/x_train_filename.npy')
y_train = np.load('/content/y_train_labels.npy')

x_test_filename = np.load('/content/x_test_filename.npy')
y_test = np.load('/content/y_test_labels.npy')

num_rows = 224
num_columns = 224
num_channels = 3

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
class Custom_Train_Generator(keras.utils.Sequence) :
    
    def __init__(self, image_filenames, labels, batch_size) :
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
    
    def __len__(self) :
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
    def __getitem__(self, idx) :
        temp_x = []
    # loading the  chunk of filenames received in batch x,batch y
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
        print(batch_x)   
    #  taking one filename at at time, finding the actual audio, producing mfcc, append to train_x
        
        for file_name in batch_x:
            if(file_name.startswith('air' or 'met' or 'sho')):
                sampling_rate, audio_signal =wavfile.read((os.path.join(dset_train_in, file_name)))
            else:
                sampling_rate, audio_signal =wavfile.read((os.path.join(dset_train_out, file_name)))
            mfcc_feature = mfcc.mfcc(audio_signal,sampling_rate, 0.01, 0.001,numcep=20,nfilt=30,nfft = 1200, appendEnergy = True)    
            mfcc_feature = preprocessing.scale(mfcc_feature)
            temp_x.append(mfcc_feature)
        temp_x = np.array(temp_x)
        #temp_x = temp_x.reshape(batch_x.shape[0], num_rows, num_columns,1)
        
        return (temp_x, np.array(batch_y))

In [11]:

class Custom_Test_Generator(keras.utils.Sequence) :
    
    def __init__(self, image_filenames, labels, batch_size) :
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
    
    def __len__(self) :
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
    def __getitem__(self, idx) :
        tempo = []
    # loading the  chunk of filenames received in batch x,batch y
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
        print(batch_x)   
    #  taking one filename at at time, finding the actual audio, producing mfcc, append to train_x
        
        for file_name in batch_x:
            if(file_name.startswith('air' or 'met' or 'sho')):
                sampling_rate, audio_signal =wavfile.read((os.path.join(dset_train_in, file_name)))
            else:
                sampling_rate, audio_signal =wavfile.read((os.path.join(dset_train_out, file_name)))
            mfcc_feature = mfcc.mfcc(audio_signal,sampling_rate, 0.01, 0.001,numcep=20,nfilt=30,nfft = 1200, appendEnergy = True)    
            mfcc_feature = preprocessing.scale(mfcc_feature)
            tempo.append(mfcc_feature)
        tempo = np.array(tempo)
        #tempo = tempo.reshape(batch_x.shape[0], num_rows, num_columns, 1)
        
        return (tempo, np.array(batch_y))

In [12]:
batch_size = 32

training_batch_generator = Custom_Train_Generator(x_train_filename, y_train, batch_size)
val_batch_generator = Custom_Test_Generator(x_test_filename, y_test, batch_size)

In [21]:
# Construct model 
num_labels= 2

from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
#restnet = Model(restnet.input, output=output)
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________

In [22]:
from keras.layers import Conv2D, MaxPooling2D , Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
model = Sequential()
model.add(restnet)
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr = 0.001),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
dense (Dense)                (None, 7, 7, 512)         1049088   
_________________________________________________________________
dropout (Dropout)            (None, 7, 7, 512)         0         
_________________________________________________________________
dense_1 (Dense)              (None, 7, 7, 512)         262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 512)         0         
_________________________________________________________________
dense_2 (Dense)              (None, 7, 7, 2)           1026      
Total params: 24,900,482
Trainable params: 1,312,770
Non-trainable params: 23,587,712
____________________________________

In [28]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 


epochs = 2
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/best_weights.pt', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit_generator(generator= training_batch_generator,
                   epochs = epochs,
                   verbose = 1,
                   validation_data = val_batch_generator,
                   callbacks=[checkpointer], )


duration = datetime.now() - start
print("Training completed in time: ", duration)

['airport-barcelona-0-0-a_1.wav' 'airport-barcelona-1-69-a_1.wav'
 'airport-barcelona-1-69-a_2.wav' 'airport-barcelona-1-70-a_1.wav'
 'airport-barcelona-1-70-a_2.wav' 'airport-barcelona-1-71-a_1.wav'
 'airport-barcelona-1-71-a_2.wav' 'airport-barcelona-1-72-a_1.wav'
 'airport-barcelona-1-72-a_2.wav' 'airport-barcelona-1-73-a_1.wav'
 'airport-barcelona-1-73-a_2.wav' 'airport-barcelona-1-74-a_1.wav'
 'airport-barcelona-1-74-a_2.wav' 'airport-barcelona-1-75-a_1.wav'
 'airport-barcelona-1-75-a_2.wav' 'airport-barcelona-1-76-a_1.wav'
 'airport-barcelona-1-76-a_2.wav' 'airport-barcelona-1-77-a_1.wav'
 'airport-barcelona-1-77-a_2.wav' 'airport-barcelona-1-78-a_1.wav'
 'airport-barcelona-1-78-a_2.wav' 'airport-barcelona-1-79-a_1.wav'
 'airport-barcelona-1-79-a_2.wav' 'airport-barcelona-1-80-a_1.wav'
 'airport-barcelona-1-80-a_2.wav' 'airport-barcelona-1-81-a_1.wav'
 'airport-barcelona-1-81-a_2.wav' 'airport-barcelona-1-82-a_1.wav'
 'airport-barcelona-1-82-a_2.wav' 'airport-barcelona-1-83-a_1.w

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


FileNotFoundError: ignored